# Identificando y modelando relaciones entre pares de variables

![correlation](https://static.thenounproject.com/png/1569699-200.png)

> En la sesión anterior introdujimos el lenguaje de programación Python, y la librería de análisis de datos para Python **Pandas**. Con Pandas, aprendimos a:
  - Cargar datos desde archivos.
  - Manipular los datos de manera básica:
    - Obtener ciertos registros.
    - Obtener ciertas columnas.
    - Obtener registros basados en condiciones.
  - Calcular estadísticas agregadas de cada variable (media, mediana, desviación estándar).
  
> Con las estadísticas de cada variable, podemos darnos una idea del comportamiento de cada variable en particular; sin embargo, no podemos inferir relaciones de una variable con otra. En esta sesión, revisaremos métodos prácticos para identificar relaciones entre variables, y además estudiaremos un método para modelar dichas relaciones de manera lineal.
___

# 1. Coeficiente de correlación

El coeficiente de correlación mide qué tan fuerte es la relación entre dos variables. Dejaremos de lado la forma de calcularlo y nos concentraremos más en sus propiedades y la interpretación que podemos darle:

- La correlación entre dos variables cualesquiera $x,y$ satisface: $$-1\leq\rho_{xy}\leq 1.$$
- La correlación de una variable con ella misma es igual a uno (1): $$\rho_{xx}=1.$$
- La correlación es simétrica: $$\rho_{xy} = \rho_{yx}.$$

In [ ]:
# Importar numpy


In [ ]:
# Importar pyplot


- Una correlación positiva $\rho_{xy}>0$ indica que los movimientos relativos entre las variables $x,y$ van en la misma dirección.

In [ ]:
# Variables con correlación positiva


In [ ]:
# Correlación


- Una correlación negativa $\rho_{xy}<0$ indica que los movimientos relativos entre las variables $x,y$ van en direcciones opuestas.

In [ ]:
# Variables con correlación negativa


In [ ]:
# Correlación


- Una correlación nula $\rho_{xy}=0$ indica que no hay una relación aparente entre las variables $x, y$.

In [ ]:
# Variables con correlación nula


In [ ]:
# Correlación


Ya que entendimos el coeficiente de correlación, utilicémoslo para descubrir la relación que hay entre el precio de las casas y su tamaño:

In [ ]:
# Importar pandas


In [ ]:
# Cargamos datos de las casas


In [ ]:
# Gráfico precio vs. tamaño


In [ ]:
# Correlación


# 2. Regresión lineal con una variable

Una vez hemos identificado la relación entre dos variables, ¿Cómo podemos modelar esta relación?

La respuesta es: **regresión lineal**.

En términos simples, el objetivo de la regresión lineal es encontrar un modelo de una recta:

$$
y = m x + b
$$

que **"mejor" (en el sentido de mínimos cuadrados) se ajuste a los puntos**. En otras palabras, el objetivo es encontrar el modelo de una recta que "aprenda" los datos.

Matemáticamente, si los puntos son

$$
\{(x_1, y_1), (x_2, y_2), \dots, (x_n, y_n)\},
$$

lo que queremos es estimar los valores de los parámetros $m$ y $b$ que minimizan la siguiente función:

$$
J(m, b) = \sum_{i=1}^{n}(y_i - (m x_i + b))^2
$$

### ¿Cómo hacemos esto en python?

Mediante la clase `LinearRegression` de la librería `sklearn`:

In [ ]:
# Importar sklearn.linear_model.LinearRegression


In [ ]:
# Ajustar la mejor recta a los datos de tamaños y precios


In [ ]:
# Obtener parámetros ajustados


In [ ]:
# Gráfico de datos y de modelo ajustado


### ¿Y esto para qué nos sirve? Sistema automático de avalúos

Supongamos que un amigo nuestro tiene una casa de $2000$ pies cuadrados en Portland, y desea saber a qué precio la podría vender. Se acaba de enterar que justo acabamos de generar un sistema automático que determina el precio de las casas de Portland usando el tamaño de las mismas.

¿A qué precio podría vender nuestro amigo la casa?

In [ ]:
# Determinar el precio


In [ ]:
# Gráfico
